# Inferential statistics
## Part III - Inferential Analysis

We're now going to look for answers to the ongoing basketball discussions between you and your family. The main ones we want to reasearch are the following:

- Your grandmother says that your sister couldn't play in a professional basketball league (not only the WNBA, but ANY professional basketball league) because she's too skinny and lacks muscle.
- Your sister says that most female professional players fail their free throws.
- Your brother-in-law heard on the TV that the average assists among NBA (male) and WNBA (female) players is 52 for the 2016-2017 season. He is convinced this average would be higher if we only considered the players from the WNBA.

Let's investigate these claims and see if we can find proof to refute or support them.

### Libraries
Import the necessary libraries first.

In [14]:
# Libraries
import math
import pandas as pd
import numpy as np
from scipy import stats as st
import matplotlib.pyplot as plt
from scipy.stats import ttest_1samp
pd.set_option('display.max_columns', 50)

### Load the dataset

Load the cleaned dataset.

In [15]:
#your code here
wnba=pd.read_csv(r"..\data\wnba_clean.csv")
wnba.head(2)

,Name,Team,Pos,Height,Weight,BMI,Birth_Place,Birthdate,Age,College,Experience,Games Played,MIN,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TO,PTS,DD2,TD3
0,Aerial Powers,DAL,F,183,71,21.200991,US,"January 17, 1994",23,Michigan State,2,8,173,30,85,35.3,12,32,37.5,21,26,80.8,6,22,28,12,3,6,12,93,0,0
1,Alana Beard,LA,G/F,185,73,21.329438,US,"May 14, 1982",35,Duke,12,30,947,90,177,50.8,5,18,27.8,32,41,78.0,19,82,101,72,63,13,40,217,0,0


# Question 1: Can my sister play in a professional female basketball league?

As we said, you grandmother is convinced that your sister couldn't play in a professional league because of her physique and weight (her weight is 67kg). 

To find an actual answer to the question we first need to know what's the average weight of a professional female basketball player. The data we have only refers to the WNBA league and not to every female professional basketball league in the world, therefore we have no way of actually calculating it.

Still, given that we do have *some* data we can **infer** it using a sample of players like the one we have. 

**How would you do it? Try and think about the requirements that your sample must satisfy in order to be used to infer the average weight. Do you feel it actually fulfills those requirements? Do you need to make any assumptions? We could calculate a confidence interval to do the inference, but do you know any other ways?**

In [16]:
# your answer here


**Now that all the requirements have been taken into account, compute the confidence interval of the average weight with a confidence level of 95%.**

In [17]:
# your code here
mean=wnba['Weight'].mean()
s=wnba['Weight'].std(ddof=1)
n=len(wnba)
c=0.99
t=st.t.ppf(1-(1-c)/2,n-1)
st.t.interval(c,n-1,loc=mean,scale=s/np.sqrt(n))


(76.56937689558555, 81.3883695832877)

**What can you say about these results?**

In [18]:
#your-answer-here
''' my grandma is always right about my sister participating in wnba, because she doenst belong in the range of 99% confidence level'''

' my grandma is always right about my sister participating in wnba, because she doenst belong in the range of 99% confidence level'

**If your sister weighs 67kg what would you tell your grandmother in regards to her assumption?**

In [19]:
#your-answer-here
"my grandma is always right about my sister participating in wnba"

'my grandma is always right about my sister participating in wnba'

## Bonus: Can you plot the probability distribution of the average weight, indicating where the critical region is?

In [20]:
# your code here

# Question 2: Do female professional basketball players fail the majority of their free throws?

You do not agree with your sister when she says that most female players fail their free throws. You decide to try and estimate the percentage of players that fail more than 40% of their free throws using, you guessed it, the WNBA sample.

**How would you do it? Try and think about the requirements that your sample must satisfy in order to be used to infer the proportion of players that miss more than 40% of their free throws. Do you feel it actually fulfills those requirements? Do you need to make any assumptions?**

In [47]:
# your answer here

p=len(wnba[wnba['FT%']<60])/len(wnba)
print(p*100)


9.859154929577464


**Now that all the requirements have been taken into account, compute the confidence interval of the proportion with a confidence level of 95%:**

In [49]:
# your code here
mean=wnba['FT%'].mean()
s=wnba['FT%'].std(ddof=1)
n=len(wnba)
c=0.95
t=st.t.ppf(1-(1-c)/2,n-1)
st.t.interval(c,n-1,loc=mean,scale=s/np.sqrt(n))


(72.75371835469808, 78.90402812417513)

**What can you comment about our result? What would you tell your sister?**

In [50]:
#your-answer-here
''' we are 95% confident that the mean of wnba FT% is between 73% & 79%'''

' we are 95% confident that the mean of wnba FT% is between 73% & 79%'

# Bonus: Can you plot the probability distribution of the proportion of missed free throws, indicating where the critical region is?

In [24]:
#your code here

# Question 3: Is the average number of assists for WNBA players only higher than the average for WNBA and NBA players together?

Your brother-in-law is convinced that the average assists for female professional players is higher than the average of both female and male players combined (which is 52 for the 2016-2017 season). You would like to actually prove if this is true or not but you remember your stats teacher saying "you can't *prove* anything, you just can say that *you are not* saying foolishness".

**How would you do it? Try and think about the requirements that your sample must satisfy in order to do that. Do you feel it actually fulfills those requirements? Do you need to make any assumptions?**

In [61]:
#your-answer-here
wnba['AST'].mean()

44.514084507042256

**Use a two-tailed one-sample t-test to see if we can reject (or not) the null hypothesis with a 95% confidence level.**

In [60]:
#your code here

## 1 step:
''' H0: wnba AST mean =52
    H1: wnba AST mean =! 52 '''

### 2 step: choose significant level
''' signficant level 5%'''
alpha=0.05

## 3 step: sample (we are using the same sample as above)

wnba_ast=wnba['AST']


stat, pvalue=st.ttest_1samp(wnba_ast,52)
print(pvalue<alpha)
stat


True


-2.1499947192482898

In [27]:
#your-answer-here

''' the pvalue is smaller than alpha, we reject the H0
in other term, the average wnba AST  is lower than the NBA+WNBA by 2 standard deviation'''

**Now use a one-tailed one-sample t-test to see if we can reject (or not) the null hypothesis with a 95% confidence level.**

In [53]:
#your-answer-here

## 1 step:
''' H0: wnba AST mean <=52
    H1: wnba AST mean > 52 '''

### 2 step: choose significant level
''' signficant level 5%'''
alpha=0.05

## 3 step: sample (we are using the same sample as above)

wnba_ast=wnba['AST']


st.ttest_1samp(wnba_ast,52, alternative='greater')

Ttest_1sampResult(statistic=-2.1499947192482898, pvalue=0.9833692293229463)

In [ ]:
'''' I THINK THAT MY BROTTHER IN LAW IS NOT RIGHT,
 BASED ON MY STUDY WITH A 95% CONFIDENCE THAT THE WNBA AST IS LOWER OR EQUAL TO THE 52 AVERAGE NBA+WNBA'''

# Bonus: Can you plot the resulting t-distribution of both tests? Indicate where the is the critical region and where does your statistic fall.**

In [29]:
#your code here

# Bonus: Satisfying your curiosity

You finally managed to solve your family's debates over basketball! While you were doing that you started to take an interest in the normal distribution.

You read that the normal distribution is present in a lot of natural phenomenons, like blood pressure, IQ, weight and height. If, for example, we could plot the distribution of the weights of every human on the planet right now it would have the shape of a normal distribution.

In light of this you would like to see if it's possible to check if the distribution of the weights of the WNBA players is a sample distribution that comes from a population that has a normal distribution, because theoretically this should be the case.

**How would you try to demonstrate that our sample fits a normal distribution? What kind of test would you use? Would you have to make any assumptions?**

In [30]:
#your-answer-here

In [31]:
# your code here

**What are your comments in regards to the results of the test?**

In [32]:
#your-answer-here